In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split

This is an example from https://www.freecodecamp.org/news/how-to-extract-keywords-from-text-with-tf-idf-and-pythons-scikit-learn-b2a0f3d7e667/

In [2]:
df_idf = pd.read_csv('amazon/reviews.csv')

In [3]:
print("Schema:\n", df_idf.dtypes)
print("Shape of database =", df_idf.shape)

Schema:
 asin             object
name             object
rating            int64
date             object
verified           bool
title            object
body             object
helpfulVotes    float64
dtype: object
Shape of database = (82815, 8)


In [4]:
def pre_process(text):
    # to lowercase
    text=text.lower()
    
    # remove tags
    text = re.sub("&lt;/?.*?&gt;", "&lt;&gt; ", text)
    
    # remove special characters and digits
    text = re.sub("(\\d|\\W)+", " ", text)
    
    return text

In [5]:
df_idf['text'] = df_idf['title'] + " " + df_idf['body']
df_idf['text'] = df_idf['text'].apply(lambda x: pre_process(str(x)))

In [6]:
df_idf['text'][2]

'love this phone this is a great reliable phone i also purchased this phone after my samsung a died the menu is easily comprehendable and speed dialing is available for around numbers voice dialing is also a nice feature but it takes longer than speed dialing the only thing that bothers me is the games nokia seems to have taken snake and off their phones there is a skydiving game bowling and tennis like pong the ringers are very nice and a feature is available to choose a different ringer for each person calling however ringtones are not available online to download to this phone you re pretty much stuck with what you have there are vibrating ringtones and regular midi polyphonic tones all they need are covers in a reasonable price range '

In [7]:
def get_stop_words(stop_file_path):
    with open(stop_file_path, 'r', encoding='utf-8') as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

In [8]:
stopwords = get_stop_words('stopwords.txt')
docs = df_idf['text'].tolist()

In [9]:
cv = CountVectorizer(max_df = .85, stop_words=stopwords)
wordCountVec = cv.fit_transform(docs)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [10]:
list(cv.vocabulary_.keys())[:10]

['def',
 'best',
 'worst',
 'samsung',
 'awhile',
 'absolute',
 'doo',
 'read',
 'review',
 'detect']

In [11]:
tfidf = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf.fit(wordCountVec)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [12]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1] , x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    sorted_items = sorted_items[:topn]
    
    score_vals = []
    feature_vals = []
    
    for idx, score in sorted_items:
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
        
        results = {}
        for idx in range(len(feature_vals)):
            results[feature_vals[idx]]=score_vals[idx]
            
        return results

In [13]:
feature_names = cv.get_feature_names()

doc = docs[1]

tf_idf_vector = tfidf.transform(cv.transform([doc]))

sorted_items = sort_coo(tf_idf_vector.tocoo())

keywords = extract_topn_from_vector(feature_names, sorted_items, 10)


In [14]:
for idx in range(len(sorted_items)):
    print(feature_names[sorted_items[idx][0]], sorted_items[idx][1])

sprint 0.4422624811530375
server 0.27322430080030485
messaging 0.2222996727555999
text 0.16275623135017
nokia 0.16238306895070545
award 0.16196262283428703
work 0.1617242655124987
winning 0.16042231072969274
retrieved 0.16042231072969274
compose 0.16042231072969274
inbox 0.15772254215033565
software 0.15456950594039445
finest 0.1507606400589547
innovation 0.14715440708513092
viewed 0.14532289710230942
burning 0.14420790020166097
offline 0.1356323931589845
hook 0.1281559136036036
patch 0.12776172223209514
team 0.1257504127182296
reply 0.12321012942987197
providers 0.12014165181651874
admit 0.11809031892330436
incoming 0.11627028016656353
capabilities 0.11245502429663828
designed 0.10869845493640888
respond 0.10646580109971827
likes 0.10535775668143048
connected 0.09731789526124357
stay 0.09386631461589888
wait 0.09291470760828649
hour 0.09269086684154547
messages 0.09244817081984441
spend 0.0921444643922326
check 0.09096369653732614
making 0.08977031552519807
due 0.08580807374985736
lea

In [15]:
y = df_idf['rating']
# fixing the labels, if > 3.5 is going to be 1 which is positive, else 0
y = y.apply(lambda x: 1 if x > 3.5 else 0) 
x = wordCountVec.toarray()
print(x.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(82815, 34848) (82815,)
(66252, 34848) (66252,)
(16563, 34848) (16563,)


In [1]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)

NameError: name 'X_train' is not defined